# Chapter05. Deep Learning Computation

Block의 개념

![](https://d2l.ai/_images/blocks.svg)

In [9]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X) # net.__call__(X) 와 동일

tensor([[ 0.1126, -0.1271,  0.0306, -0.0439, -0.2753,  0.0415, -0.1361,  0.1953,
          0.0368,  0.0960],
        [ 0.1482, -0.0982,  0.0397, -0.1091, -0.3710,  0.0402, -0.1852,  0.0734,
         -0.1105,  0.0273]], grad_fn=<AddmmBackward>)

In [10]:
# net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) 
# 이거를 풀어쓰면 아래 클래스로 구성된 것

class MLP(nn.Module):
    
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

* \_\_init\_\_ 부분에서는 사용할 레이어를 만들어주고 파라미터 초기화를 해줌(나중 다룸)
* forward에서 input X에 대해서 계산이 어떻게 이루어지는지 설정 (모델 설정)

In [11]:
net = MLP()
net(X)

tensor([[ 0.1124,  0.0619, -0.0630,  0.2029, -0.1664,  0.0257,  0.4505, -0.0307,
          0.0889,  0.1716],
        [ 0.2236,  0.1355, -0.0356,  0.2879, -0.0520, -0.0578,  0.5374,  0.0347,
          0.0100,  0.1822]], grad_fn=<AddmmBackward>)

## Sequential Block

In [25]:
# nn.Sequential이 이런 구조로 만들어져 있음

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [26]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) 

In [27]:
net(X)

tensor([[ 0.1955,  0.0426, -0.0157, -0.2321,  0.0570,  0.0278,  0.0875,  0.0161,
         -0.2953,  0.2590],
        [ 0.1574, -0.0241,  0.0784, -0.1834,  0.0206, -0.0396,  0.0613,  0.2622,
         -0.2222,  0.3335]], grad_fn=<AddmmBackward>)

In [28]:
args = nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)

## Fixed Hidden MLP

In [31]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

`self.rand_weight`는 모델 파라미터가 아님 (requires_grad=False), 그냥 fixed tensor

In [34]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0426, grad_fn=<SumBackward0>)

위에서 만든 것들 섞어보기

In [36]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1482, grad_fn=<SumBackward0>)

Summary
* Layers are blocks.

* Many layers can comprise a block.

* Many blocks can comprise a block.

* A block can contain code.

* Blocks take care of lots of housekeeping, including parameter initialization and backpropagation.

* Sequential concatenations of layers and blocks are handled by the Sequential block.

# Parameter Management

In [37]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1413],
        [0.0583]], grad_fn=<AddmmBackward>)

## Parameter Access

In [38]:
# nn.Linear(8,1)
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2711, -0.1918,  0.0273, -0.1990,  0.2479,  0.2222,  0.2020, -0.0088]])), ('bias', tensor([0.2209]))])


### Targeted Parameters

In [46]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [47]:
print(net[2].bias)

Parameter containing:
tensor([0.2209], requires_grad=True)


In [48]:
print(net[2].bias.data)

tensor([0.2209])


In [54]:
print(net[2].weight.data)

tensor([[-0.2711, -0.1918,  0.0273, -0.1990,  0.2479,  0.2222,  0.2020, -0.0088]])


Parameters are complex objects, containing values, gradients, and additional information. That’s why we need to request the value explicitly.

In [56]:
net[2].weight.grad == None

True

Back Prop 안했기 때문에 True

### All Parameters at Once

In [58]:
# net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [59]:
net.state_dict()['2.bias'].data

tensor([0.2209])

### Collecting Paramters from Nested Blocks

In [62]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1()) # 블록 이름, 블록
    return net

In [61]:
X

tensor([[0.5238, 0.5631, 0.8346, 0.0199],
        [0.1230, 0.0452, 0.3123, 0.9566]])

In [63]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1251],
        [-0.1250]], grad_fn=<AddmmBackward>)

In [64]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


we can access the first major block, within it the second sub-block, and within that the bias of the first layer, with as follows.

In [66]:
# 위에 숫자 인덱싱 참고
rgnet[0][1][0].bias.data

tensor([-0.1161,  0.0701, -0.3421,  0.4696,  0.4614,  0.2183, -0.3551,  0.0300])

## Initialization

By default, PyTorch initializes weight and bias matrices uniformly by drawing from a range that is computed according to the input and output dimension. PyTorch’s nn.init module provides a variety of preset initialization methods.

In [67]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0015,  0.0014,  0.0044, -0.0021]), tensor(0.))

In [68]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [69]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # Xavier initializer

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1840, -0.7053,  0.1578, -0.1005])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


[Initialization 정리 잘된 글](https://www.deeplearning.ai/ai-notes/initialization/index.html#IV)

Custom Initialization

\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}

In [70]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.9863,  0.0000,  8.9998, -0.0000],
        [-0.0000,  0.0000, -8.0048, -7.7098]], grad_fn=<SliceBackward>)

Note that we always have the option of setting parameters directly.



In [72]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  9.9998,  1.0000])

# Custom Layers

## Layers without Parameters

In [73]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [74]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [75]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [76]:
Y = net(torch.rand(4, 8))
Y.mean() # mean(X-X.mean()) => 0 나와야 함

tensor(-3.2596e-09, grad_fn=<MeanBackward0>)

## Layers with Parameters